# Execution Interval Method

In [ ]:
import json
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from collections import defaultdict
from libraries.utils import *
from libraries.exeint import exeInt




## Load Data

In [ ]:
############ configuration ################
############################################

CODE = 'theft_protection'       ### application (code)    theft_protection, mamba2, lora_ducy
BEHAVIOUR_FAULTY = 'faulty_data'            ### normal, faulty_data
BEHAVIOUR_NORMAL = 'normal'            ### normal, faulty_data
THREAD = 'single'           ### single, multi
VER = 4                     ### format of data collection

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
normalbase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_NORMAL}'
faultybase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_FAULTY}'

print(normalbase_path)
print(faultybase_path)

In [ ]:

train_base_path = os.path.join(normalbase_path, 'train_data')
train_data_path = [os.path.join(train_base_path, x) for x in os.listdir(train_base_path)]
train_varlist_path = os.listdir(normalbase_path)
train_varlist_path = [os.path.join(normalbase_path, x) for x in train_varlist_path if 'varlist' in x]

######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(faultybase_path)

### remove.Ds_store from all lists
train_data_path = [x for x in train_data_path if '.DS_Store' not in x]
train_varlist_path = [x for x in train_varlist_path if '.DS_Store' not in x]
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()
paths_label.sort()

# print(paths_log)
# print(paths_traces)
# print(varlist_path)
# print(paths_label)

test_data_path = paths_traces
test_label_path = paths_label

print(train_data_path)
print(test_data_path)
print(test_label_path)


In [ ]:
varlist_path

In [ ]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3 or VER == 4:
    check_con, _ = is_consistent([train_varlist_path[0]]+ varlist_path) ### compare with train varlist

    if check_con != False:
        to_number = read_json(varlist_path[0])
        from_number = mapint2var(to_number)
    else:
        ### load normal varlist
        print('loading normal varlist')
        to_number = read_json(train_varlist_path[0])
        from_number = mapint2var(to_number)



In [ ]:
to_number = read_json(train_varlist_path[0])
from_number = mapint2var(to_number)

In [ ]:
# #### key finder ####
# from_number[44]

In [ ]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

## Confidence Interval

__Confidence Interval:__

A confidence interval is a range around the mean that is likely to contain the true population mean. The formula for a confidence interval is mean ± margin of error mean±margin of error, where the margin of error depends on the desired confidence level and the standard error.

_Example:_

1. Choose a confidence level (e.g., 95%).
2. Calculate the standard error: standard deviation/ sqr_root(number of observations)
3. Calculate the margin of error: critical value × standard error
4. Determine the confidence interval: mean ± margin of error


In [ ]:
### initialize exeinz
ei = exeInt()

### Data Processing

In [ ]:
### get execution intervals for all variables

exe_list, filewise_exe_list = ei.get_exeint(train_data_path)

In [ ]:
for k in list(exe_list.keys()):
    print(k, len(exe_list[k]))

In [ ]:
max(exe_list[6])

In [ ]:
################## methods to detect outliers based on execution intervals ####################

############ calculate dynamic thresholds ############
thresholds = ei.get_dynamicthresh(exe_list)

############ train lof model ################
lof_models = ei.train_lof(exe_list)

######### save thresholds and lof models ############
### visualize the thresholds for varlist
thresholds_var = {}
for key in thresholds.keys():
    print('key:', key)
    thresholds_var[from_number[key]] = thresholds[key]

assert len(thresholds_var) == len(thresholds)
thresholds_var
save_json(thresholds_var, os.path.join(faultybase_path, 'thresholds.json'))

In [ ]:
from_number

In [ ]:
thresholds_var

### Window size for Diagnosis

In [ ]:
### number of events between two consecutive occurrence of each event
import scipy
import scipy.stats as stats

event_list, filewise_event_list = ei.get_eventint(train_data_path)
event_range = ei.get_eventrange(event_list)

print('event_range:', event_range)
max_range = 0
all_intervals = []
for ranges in event_range.values():
    # print('range:', max(ranges))
    all_intervals.extend(ranges)

print('intervals:', all_intervals)
max_window = max(all_intervals)
mean_window = np.mean(all_intervals)
median_window = np.median(all_intervals)
mode_window = stats.mode(all_intervals)

std_deviation = np.std(all_intervals)
# variance = np.var(all_intervals)

print('max_window:', max_window)
print('mean_window:', mean_window)
print('median_window:', median_window)
print('mode_window:', mode_window)
print('std_deviation:', std_deviation)
# print('variance:', variance)
### get max window size for diagnosis

### Visualising Thresholds

In [ ]:
### plot exe_list to vsiualize the distribution of execution intervals
# ei.viz_thresholds(exe_list, thresholds=thresholds)


## Validation

In [ ]:
#### Detect anomalies in faulty traces
DIFF_VAL = 5
all_tp = []
all_fp = []
all_fn = []
all_detections = [] ### format [file1_detection, file2_detection] -> file1_detection: [(state1, 0), (ts1, ts2), filename]  
all_group_detections = [] ### format [file1_detection, file2_detection] -> file1_detection: [(state1, 0), (ts1, ts2), filename]
all_merged_detections = [] ### format [file1_detection, file2_detection] -> file1_detection: [(state1, 0), (ts1, ts2), filename]
y_pred_all = []
y_true_all = []
all_gt = []
for test_data, test_label in zip(test_data_path, test_label_path):
    print(test_data, test_label)

    detection = ei.test_single(test_data, thresholds=thresholds)   ### detection in format: [var, (ts1,ts2), file_name]     ### threshold based detection
    print('detection:', detection)
    # detection = ei.test_single(test_data, lof_models=lof_models)   ### detection in format: [var, (ts1,ts2), file_name]    ### lof based detection
    before_merge = len(detection)

    merged_detection, grouped_det, events_exe = ei.merge_detections(detection, DIFF_VAL, get_exetime=True)  ### merge detections for multiple variables
    # merged_detection, grouped_det = ei.merge_detections(detection, DIFF_VAL)  ### merge detections for multiple variables
    detection = merged_detection
    print('merged detections:', detection)
    print('events_exe:', events_exe)

    # dedup_detection, grouped_det = ei.remove_duplicates(detection, DIFF_VAL)  ### remove multiple detections for single ground truth
    # detection = dedup_detection
    after_merge = len(detection)
    # print('before merge:', before_merge, 'after merge:', after_merge)

    # all_group_detections += [(test_data, grouped_det, test_label)]  ### used to plot grouped detections
    # all_merged_detections += [(test_data, merged_detection, test_label)]  ### used to plot merged detections

    ### load ground truths
    ground_truth_raw = read_traces(test_label)
    ground_truth = ground_truth_raw['labels']
    label_trace_name = list(ground_truth.keys())[0]
    ground_truth = ground_truth[label_trace_name]
    print('ground truths:', ground_truth)
    print(len(ground_truth))

    # correct_pred, rest_pred, y_pred, y_true = get_ypred_ytrue(detection, ground_truth)  ### case1_pred, case2_pred, case34_pred, rest_pred
    # correct_pred, rest_pred, y_pred, y_true = ei.get_correct_detections(detection, ground_truth)  ### case1_pred, case2_pred, case34_pred, rest_pred
    correct_pred, rest_pred, y_pred, y_true, false_neg = ei.get_correct_detections(detection, ground_truth)  ### case1_pred, case2_pred, case34_pred, rest_pred

    assert( len(detection) == len(correct_pred) + len(rest_pred) )

    all_detections += [(test_data, detection, test_label)]  ### used to plot detections
    all_tp += [(test_data, correct_pred, test_label)]
    all_fp += [(test_data, rest_pred, test_label)]
    all_fn += [(test_data, false_neg, test_label)]
    all_gt += [(test_data, ground_truth, test_label)]


    y_pred_all.extend(y_pred)
    y_true_all.extend(y_true)

    # break

### Detection Quality after diff_val (percent of non-anomalous trace)

In [ ]:
all_tp_quality = []
all_quality_scores = []
all_gt_count = []
all_tp_count = []
for file_tp in all_tp[0:]:
    print(file_tp)
    file_path = file_tp[0]
    print('File Path:', file_path)
    print('File Name:', os.path.basename(file_path))

    trace = read_traces(file_path)
    # print('Trace Data:', trace)
    time_stamps = [e[1] for e in trace]
    events = [e[0] for e in trace]
    # print('Time Stamps:', time_stamps)
    # print('Events:', events)

    time_stamps = np.array(time_stamps)
    events = np.array(events)

    ### load GT
    label_path = file_tp[2]
    # print('Label Path:', label_path)
    ground_truth_raw = read_traces(label_path)
    # print('Ground Truth Raw:', ground_truth_raw)
    ground_truth = ground_truth_raw['labels']
    # print('Ground Truth:', ground_truth)
    label_trace_name = list(ground_truth.keys())[0]
    ground_truth = ground_truth[label_trace_name]
    # print('Label Trace Name:', label_trace_name)
    # print('Ground Truth Length:', len(ground_truth))
    print('Ground Truth:', ground_truth)

    quality_score = []
    gt_count = 0
    tp_count = 0
    for tp in file_tp[1]:
        print('TP:', tp)
        pd_ts1 = tp[1][0]
        pd_ts2 = tp[1][1]
        # print('Detection Start:', pd_ts1)
        # print('Detection End:', pd_ts2)

        ### get the closest timestamp to pd_ts1 and pd_ts2
        start_tp = np.argmin(np.abs(time_stamps - pd_ts1))
        end_tp = np.argmin(np.abs(time_stamps - pd_ts2))
        print('Start Index:', start_tp)
        print('End Index:', end_tp)
        
        ### get exact timestamp match for pd_ts1 and pd_ts2
        # start_tp = np.where(time_stamps == pd_ts1)[0][0]
        # end_tp = np.where(time_stamps == pd_ts2)[0][0]
        # print('Start Index:', start_index)
        # print('End Index:', end_index)

        ### get the sub trace corresponding to the detection
        det_trace = trace[start_tp:end_tp+1]
        len_det_trace = len(det_trace)
        # print('Detection Trace:', det_trace)
        # print('Detection Trace Length:', len(det_trace))

        ### collect groundtruths that intersect with the detection
        gt_overlap_trace = []
        sel_gt = []
        for gt in ground_truth:
            gt_ts1 = gt[2]
            gt_ts2 = gt[3]
            # print('Ground Truth Start:', gt_ts1)
            # print('Ground Truth End:', gt_ts2)
            cond_1 = pd_ts1 >= gt_ts1 and pd_ts2 <= gt_ts2  ### check if the detection timestamp is within the ground truth timestamp (case 1)
            cond_2 = pd_ts1 <= gt_ts1 and pd_ts2 >= gt_ts2  ### check if the gorund truth timestamp is within the detection timestamp (case 2)
            cond_3 = pd_ts1 >= gt_ts1 and pd_ts1 <= gt_ts2 and pd_ts2 >= gt_ts2    ### partial detection on right of the ground truths, check 5 second difference after this (case 3)
            cond_4 = pd_ts2 <= gt_ts2 and pd_ts2 >= gt_ts1 and pd_ts1 <= gt_ts1   ### partial detection on left of the ground truths, check 5 second difference after this (case 4)

            if cond_1 or cond_2 or cond_3 or cond_4:
                print('GT', gt)
                if gt not in sel_gt:
                    sel_gt.append(gt)

                start_gt = np.where(time_stamps == gt_ts1)[0][0]
                end_gt = np.where(time_stamps == gt_ts2)[0][0]
                # print('Start Index GT:', start_gt)
                # print('End Index GT:', end_gt)
                gt_trace = trace[start_gt:end_gt+1]
                # print('Ground Truth Trace:', gt_trace)
                # print('len of gt:', len(gt_trace))

                for (gt_e, gt_t) in gt_trace:
                    if gt_t >= pd_ts1 and gt_t <= pd_ts2:
                        gt_overlap_trace.append((gt_e, gt_t))


        print('Selected Ground Truths:', sel_gt)
        # print('gt overlap_trace:', gt_overlap_trace)

        ### calculate quality of detection (percentge of detection trace that do not overlap with ground truth)
        if len(gt_overlap_trace) == 0:
            # raise ValueError('No ground truth overlap trace found for detection', tp, ', file path:', file_path)
            overlap_percentage = 0.0
        else:
            overlap_percentage = len(gt_overlap_trace) / len_det_trace    ### precision of detection (point-wise)
            overlap_percentage = np.round(overlap_percentage, 2)


            # nonanomaly_percentage = 1 - overlap_percentage
            # nonanomaly_percentage = np.round(nonanomaly_percentage, 2)
            # print('Detection with normal trace (Percentage):', nonanomaly_percentage)
            # quality_score.append(nonanomaly_percentage)
            
        print('Point-wise precision of detection:', overlap_percentage)
        quality_score.append(overlap_percentage)

        
        ### calculate quality metric (detection to ground truth ratio)
        gt_count = len(sel_gt)
        tp_count += 1

        print('')
        # break    

    all_gt_count += [gt_count]
    all_tp_count += [tp_count]
    
    all_tp_quality.append((file_path, quality_score, label_path))
    all_quality_scores.extend(quality_score)
    print('GT count:', gt_count)
    print('TP count:', tp_count)
    print('')   
    # break

print('Total GT count:', all_gt_count)
print('Total TP count:', all_tp_count)

In [ ]:
all_quality_scores = np.array(all_quality_scores)
print('All quality scores:', all_quality_scores)
print('Average quality score:', np.mean(all_quality_scores))
print('Standard deviation of quality scores:', np.std(all_quality_scores))

In [ ]:
all_quality_scores = np.array(all_quality_scores)
print('All quality scores:', all_quality_scores)
print('Average quality score:', np.mean(all_quality_scores))
print('Standard deviation of quality scores:', np.std(all_quality_scores))

In [ ]:
#### Caculate quality score for each file as well as overall quality score
overall_quality_score = []
gtc = 0
tpc = 0
for file_quality, gt_count, tp_count in zip(all_tp_quality, all_gt_count, all_tp_count):
    file_path = file_quality[0]
    quality_score = file_quality[1]
    label_path = file_quality[2]

    print('File Path:', file_path)
    # print('Quality Score:', quality_score)
    # print('Label Path:', label_path)

    if len(quality_score) == 0:
        print('No quality score found for file:', file_path)
        continue

    avg_quality = np.mean(quality_score)
    avg_quality = np.round(avg_quality, 2)
    std_quality = np.std(quality_score)
    std_quality = np.round(std_quality, 2)
    overall_quality_score.extend(quality_score)
    print('Average Quality Score for File:', avg_quality, '±', std_quality)

    ### calculate quality metric (detection to ground truth ratio)
    per_file = tp_count / gt_count
    print('Detection to Ground Truth Ratio for File:', per_file)

    gtc += gt_count
    tpc += tp_count

    print('')

overall_avg = np.mean(overall_quality_score)
overall_std = np.std(overall_quality_score)
overall_avg = np.round(overall_avg, 2)
overall_std = np.round(overall_std, 2)
print('Overall Average Quality Score:', overall_avg, '±', overall_std)
print('')

overall_tgr = tpc / gtc
overall_tgr = np.round(overall_tgr, 2)
print('Overall Detection to Ground Truth Ratio:', overall_tgr)



# print('Total Ground Truth Count:', all_gt_count)
# print('Total True Positive Count:', all_tp_count)
# det2gt = all_tp_count / all_gt_count
# det2gt = np.round(det2gt, 2)
# print('Detection to Ground Truth Ratio:', det2gt)

In [ ]:
print(all_fp)
fp_count = 0
for fp in all_fp:
    print( len(fp[1]))
    fp_count += len(fp[1])

print('fp_count:', fp_count)

In [ ]:
### Evaluation metrics

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, average_precision_score, ConfusionMatrixDisplay


# Calculate precision
precision = precision_score(y_true_all, y_pred_all)
print(f'Precision: {precision:.4f}')

# Calculate recall
recall = recall_score(y_true_all, y_pred_all)
print(f'Recall: {recall:.4f}')

# # Calculate average precision
# average_precision = average_precision_score(y_true_all, y_pred_all)
# print(f'Average Precision: {average_precision:.4f}')

# Calculate F1 score
f1 = f1_score(y_true_all, y_pred_all)
print(f"F1 Score: {f1:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true_all, y_pred_all)
print("Confusion Matrix:")
print(conf_matrix)
if len(conf_matrix) == 1:
    conf_matrix = np.array([[0, 0], [0, conf_matrix[0][0]]])
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['normal', 'anomaly'])
disp.plot()

## Classwise Detections

In [ ]:
classwise_fn = defaultdict(list)
classwise_tp = defaultdict(list)
gt_len = 0
for file_fn, file_gt in zip(all_fn, all_gt):
    fn = file_fn[1]
    gt = file_gt[1]
    for label in gt:
        if label in fn:
            classwise_fn[label[4]].append(label)
        else:
            classwise_tp[label[4]].append(label)
            # print('tp:', label)

    gt_len += len(gt)
    # print('file gt:', len(gt))
    # print('file fn:', len(fn))
    # print('\n')
    # break

total_fn = 0
total_tp = 0
keys = set(list(classwise_fn.keys()) + list(classwise_tp.keys()))
# print('keys:', keys)
class_recall = []
for key in keys:
    print('class:', key)
    total_fn += len(classwise_fn[key])
    total_tp += len(classwise_tp[key])

    crecall = len(classwise_tp[key])/(len(classwise_fn[key])+len(classwise_tp[key]))

    # print('not detected:', len(classwise_fn[key]))
    print('detected:', len(classwise_tp[key]))
    print('total anomalies:', len(classwise_fn[key])+len(classwise_tp[key]))
    print('Recall (classwise):', crecall)
    print('\n')

    class_recall.append(crecall)


# print('total fn+tp:', total_fn+total_tp)
# print('total gt:', gt_len)
assert total_fn+total_tp == gt_len, 'total fn+tp not equal to total gt'
print('All class recalls:', class_recall)

## Save Detections

In [ ]:
######## save detections for the dashboard to plot #############
import traceback

for test_data, detections, test_label in all_detections:
    # print(test_data, test_label)
    # print(test_label.replace('labels', 'detections'))
    detection_path = test_label.replace('labels', f'ei_detections')
    detection_path = detection_path.replace('ei_detections.json', f'ei_detections_{DIFF_VAL}.json')
    # tp_detection_path = detection_path.replace('ei_detections.json', f'tp_ei_detections_{DIFF_VAL}.json')
    # fp_detection_path = detection_path.replace('ei_detections.json', f'fp_ei_detections_{DIFF_VAL}.json')
    # print(detections)
    # print(detection_path)

    detection_dir = os.path.dirname(detection_path)
    # print(detection_dir)
    if not os.path.exists(detection_dir):
        os.makedirs(detection_dir)
        print(f'Created Directory: {detection_dir}')

    try:
        with open(detection_path, 'w') as f:
            json.dump(detections, f)
            print(f'Saved detections in {detection_path}')

            
    except Exception as e:
        traceback.print_exception(e)
        print('Error in saving detections')
        continue

for test_data, detections, test_label in all_tp:
    # print(test_data, test_label)
    # print(test_label.replace('labels', 'detections'))
    detection_path = test_label.replace('labels', 'ei_detections')
    tp_detection_path = detection_path.replace('ei_detections.json', f'tp_ei_detections_{DIFF_VAL}.json')
    # fp_detection_path = detection_path.replace('ei_detections.json', 'fp_ei_detections.json')
    # print(detections)

    detection_dir = os.path.dirname(detection_path)
    # print(detection_dir)
    if not os.path.exists(detection_dir):
        os.makedirs(detection_dir)
        print(f'Created Directory: {detection_dir}')

    try:

        with open(tp_detection_path, 'w') as f:
            json.dump(detections, f)
            print(f'Saved detections in {tp_detection_path}')
            
    except Exception as e:
        traceback.print_exception(e)
        print('Error in saving detections')
        continue

for test_data, detections, test_label in all_fp:
    # print(test_data, test_label)
    # print(test_label.replace('labels', 'detections'))
    detection_path = test_label.replace('labels', 'ei_detections')
    # tp_detection_path = detection_path.replace('ei_detections.json', 'tp_ei_detections.json')
    fp_detection_path = detection_path.replace('ei_detections.json', f'fp_ei_detections_{DIFF_VAL}.json')
    # print(detections)

    detection_dir = os.path.dirname(detection_path)
    # print(detection_dir)
    if not os.path.exists(detection_dir):
        os.makedirs(detection_dir)
        print(f'Created Directory: {detection_dir}')

    try:

        with open(fp_detection_path, 'w') as f:
            json.dump(detections, f)
            print(f'Saved detections in {fp_detection_path}')
            
    except Exception as e:
        traceback.print_exception(e)
        print('Error in saving detections')
        continue

In [ ]:
# print('Total Detections:', len(all_detections[1][1]))
# print('Total Groups:', len(all_group_detections[1][1]))
# i=0
# for item in all_group_detections[1][1]:
#     for ind_item in item:
#         i+=1
# print('Detections in Groups:', i)
# print('Total Merged:', len(all_merged_detections[1][1]))

## Plot Detections

In [ ]:
# ### plot gt and detections
# for test_data, detections, test_label in all_detections:
# # for test_data, detections, test_label in all_fp:
#     # print('test_data:', test_data)
#     # print('detections:', detections)
#     # print(test_label)

#     ### prepare trace to plot
#     col_data = preprocess_traces([test_data])
#     all_df = get_dataframe(col_data) 
#     # print(all_df[0])

#     ### prepare detections to plot
#     timestamps = col_data[0][1]
#     print('timestamps:', timestamps)
#     plot_val = []
#     plot_x_ticks = []
#     plot_class = []
#     for det in detections:
#         # print(det)
#         det_ts1, det_ts2 = det[1]
#         # print(det_ts1, det_ts2)

#         det_ind1_pre = [ abs(t-det_ts1) for t in timestamps]
#         det_ind1 = det_ind1_pre.index(min(det_ind1_pre))

#         det_ind2_pre = [ abs(t-det_ts2) for t in timestamps]
#         det_ind2 = det_ind2_pre.index(min(det_ind2_pre))
#         # print(det_ind1, det_ind2)
#         # print(timestamps[det_ind1], timestamps[det_ind2])

#         plot_val += [(det_ind1, det_ind2)]
#         plot_x_ticks += [(timestamps[det_ind1], timestamps[det_ind2])]
#         plot_class += [0]

#     plot_detections = [plot_val, plot_x_ticks, plot_class]

#     ### get ground truths
#     gt_plot = prepare_gt(test_label)

#     ### plot
#     for df in all_df:
#         # print(df.columns)
#         plot_fig = plot_single_trace(df, 
#                           var_list, 
#                           with_time=False, 
#                           is_xticks=True, 
#                           detections=plot_detections, 
#                           dt_classlist=['detection'],
#                           ground_truths=gt_plot,
#                           gt_classlist=['gt_communication', 'gt_sensor', 'gt_bitflip'],
#                           )
#         plot_fig.show()

#     # break

In [ ]:
# ##### plot merged detections
# ### plot gt and detections
# # for test_data, detections, test_label in all_detections:
# # for test_data, detections, test_label in all_merged_detections: #### all merged detections
# for test_data, detections, test_label in all_fp:
#     # print('test_data:', test_data)
#     # print('detections:', detections)
#     # print(test_label)

#     ### prepare trace to plot
#     col_data = preprocess_traces([test_data])
#     all_df = get_dataframe(col_data) 
#     # print(all_df[0])

#     ### prepare detections to plot
#     timestamps = col_data[0][1]
#     print('timestamps:', timestamps)
#     plot_val = []
#     plot_x_ticks = []
#     plot_class = []
#     for det in detections:
#         # print(det)
#         det_ts1, det_ts2 = det[1]
#         # print(det_ts1, det_ts2)

#         det_ind1_pre = [ abs(t-det_ts1) for t in timestamps]
#         det_ind1 = det_ind1_pre.index(min(det_ind1_pre))

#         det_ind2_pre = [ abs(t-det_ts2) for t in timestamps]
#         det_ind2 = det_ind2_pre.index(min(det_ind2_pre))
#         # print(det_ind1, det_ind2)
#         # print(timestamps[det_ind1], timestamps[det_ind2])

#         plot_val += [(det_ind1, det_ind2)]
#         plot_x_ticks += [(timestamps[det_ind1], timestamps[det_ind2])]
#         plot_class += [0]

#     plot_detections = [plot_val, plot_x_ticks, plot_class]

#     ### get ground truths
#     gt_plot = prepare_gt(test_label)

#     ### plot
#     for df in all_df:
#         # print(df.columns)
#         plot_fig = plot_single_trace(df, 
#                           var_list, 
#                           with_time=False, 
#                           is_xticks=True, 
#                           detections=plot_detections, 
#                           dt_classlist=['detection'],
#                           ground_truths=gt_plot,
#                           gt_classlist=['gt_communication', 'gt_sensor', 'gt_bitflip'],
#                           )
#         plot_fig.show()

#     # break

In [ ]:
1890

## Diff Val for paper

In [ ]:
#### diff_val for paper

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, average_precision_score, ConfusionMatrixDisplay
import pandas as pd
import traceback


diff_val_res = []  ### used to store results for different diff_val
for DIFF_VAL in range(0, 180):
    if (DIFF_VAL % 5 == 0 and DIFF_VAL <= 50) or DIFF_VAL <= 5:
        print('DIFF_VAL:', DIFF_VAL)

        # DIFF_VAL = 2
        #### Validate model
        all_detections = []  ### format [file1_detection, file2_detection] -> file1_detection: [(state1, state2), (ts1, ts2), filename]
        y_pred_all = []
        y_true_all = []
        all_tp = []
        all_fp = []
        all_fn = []
        all_gt = []

        # tp = 0
        # fp = 0
        # fn = 0
            
        for test_data, test_label in zip(test_data_path, test_label_path):
            print('Testing data:', test_data)
            detection = ei.test_single(test_data, thresholds=thresholds)   ### detection in format: [var, (ts1,ts2), file_name]     ### threshold based detection
            # print('Detections:', detection)
            print('len of detections:', len(detection))

            merged_detection, grouped_det, events_exe = ei.merge_detections(detection, DIFF_VAL, get_exetime=True)  ### merge detections for multiple variables
            detection = merged_detection


            ground_truth_raw = read_traces(test_label)
            ground_truth = ground_truth_raw['labels']
            label_trace_name = list(ground_truth.keys())[0]
            ground_truth = ground_truth[label_trace_name]
            # print('ground truths:', ground_truth)
            # print(len(ground_truth))
            print('merged detections', len(merged_detection))

            correct_pred, rest_pred, y_pred, y_true, false_neg = ei.get_correct_detections(detection, ground_truth)  ### case1_pred, case2_pred, case34_pred, rest_pred
            
            y_pred_all.extend(y_pred)
            y_true_all.extend(y_true)

            all_detections += [(test_data, detection, test_label)]  ### used to plot detections
            all_tp += [(test_data, correct_pred, test_label)]
            all_fp += [(test_data, rest_pred, test_label)]
            all_fn += [(test_data, false_neg, test_label)]
            all_gt += [(test_data, ground_truth, test_label)]


        # # Calculate precision
        # precision = precision_score(y_true_all, y_pred_all)
        # print(f'Precision: {precision:.4f}')

        # # Calculate recall
        # recall = recall_score(y_true_all, y_pred_all)
        # print(f'Recall: {recall:.4f}')

        # # Calculate average precision
        # average_precision = average_precision_score(y_true_all, y_pred_all)
        # print(f'Average Precision: {average_precision:.4f}')

        # Calculate F1 score
        f1 = f1_score(y_true_all, y_pred_all)
        # print(f"F1 Score: {f1:.4f}")

        # Calculate confusion matrix
        conf_matrix = confusion_matrix(y_true_all, y_pred_all)
        # print("Confusion Matrix:")

        ### calculate detection quality
        quality_score, total_tp_count, total_gt_count = detection_quality(all_tp, output_score=True)

        ### metric 1 (percent of non-anomalous trace)
        avg_quality_score = np.mean(quality_score)
        std_quality_score = np.std(quality_score)

        print('Detection Quality Results:', quality_score, '±', std_quality_score)

        ### metric 2 (ratio of TP to GT)
        det2gt = total_tp_count / total_gt_count

        f1 = np.round(f1, 2)
        avg_quality_score = np.round(avg_quality_score, 2)
        std_quality_score = np.round(std_quality_score, 2)
        det2gt = np.round(det2gt, 2)

        # print(conf_matrix.shape)
        if conf_matrix.shape[0] == 1:
            print(conf_matrix)
            diff_val_res.append([DIFF_VAL, f1, 0, 0, avg_quality_score, std_quality_score, det2gt])
        else:
            diff_val_res.append([DIFF_VAL, f1, conf_matrix[0][1], conf_matrix[1][0], avg_quality_score, std_quality_score, det2gt])


        ################## Saving detections ######################
        for test_data, detections, test_label in all_detections:
            # print(test_data, test_label)
            # print(test_label.replace('labels', 'detections'))
            detection_path = test_label.replace('labels', f'ei_detections')
            detection_path = detection_path.replace('ei_detections.json', f'ei_detections_{DIFF_VAL}.json')
            # tp_detection_path = detection_path.replace('ei_detections.json', f'tp_ei_detections_{DIFF_VAL}.json')
            # fp_detection_path = detection_path.replace('ei_detections.json', f'fp_ei_detections_{DIFF_VAL}.json')
            # print(detections)
            # print(detection_path)

            detection_dir = os.path.dirname(detection_path)
            # print(detection_dir)
            if not os.path.exists(detection_dir):
                os.makedirs(detection_dir)
                print(f'Created Directory: {detection_dir}')

            try:
                with open(detection_path, 'w') as f:
                    json.dump(detections, f)
                    print(f'Saved detections in {detection_path}')

                    
            except Exception as e:
                traceback.print_exception(e)
                print('Error in saving detections')
                continue

        for test_data, detections, test_label in all_tp:
            # print(test_data, test_label)
            # print(test_label.replace('labels', 'detections'))
            detection_path = test_label.replace('labels', 'ei_detections')
            tp_detection_path = detection_path.replace('ei_detections.json', f'tp_ei_detections_{DIFF_VAL}.json')
            # fp_detection_path = detection_path.replace('ei_detections.json', 'fp_ei_detections.json')
            # print(detections)

            detection_dir = os.path.dirname(detection_path)
            # print(detection_dir)
            if not os.path.exists(detection_dir):
                os.makedirs(detection_dir)
                print(f'Created Directory: {detection_dir}')

            try:

                with open(tp_detection_path, 'w') as f:
                    json.dump(detections, f)
                    print(f'Saved detections in {tp_detection_path}')
                    
            except Exception as e:
                traceback.print_exception(e)
                print('Error in saving detections')
                continue

        for test_data, detections, test_label in all_fp:
            # print(test_data, test_label)
            # print(test_label.replace('labels', 'detections'))
            detection_path = test_label.replace('labels', 'ei_detections')
            # tp_detection_path = detection_path.replace('ei_detections.json', 'tp_ei_detections.json')
            fp_detection_path = detection_path.replace('ei_detections.json', f'fp_ei_detections_{DIFF_VAL}.json')
            # print(detections)

            detection_dir = os.path.dirname(detection_path)
            # print(detection_dir)
            if not os.path.exists(detection_dir):
                os.makedirs(detection_dir)
                print(f'Created Directory: {detection_dir}')

            try:

                with open(fp_detection_path, 'w') as f:
                    json.dump(detections, f)
                    print(f'Saved detections in {fp_detection_path}')
                    
            except Exception as e:
                traceback.print_exception(e)
                print('Error in saving detections')
                continue
        ################## Saving detections ######################
        
        
        # break

df = pd.DataFrame(diff_val_res, columns=['diff_val', 'f1_score', 'FP', 'FN', 'pdPrecision', 'pdPrecStd', 'tp_gt_ratio'], index=None)
###### save df to excel
df.to_excel(f'../results/diff_val/{CODE}_ei_results.xlsx', index=False)
print(df)

Observations
---
- since multiple variables are affected due to single anomaly, multiple detections are generated for each anomaly.
- This leads to multiple FP.
- To avoid this, we implement deduplication which groups the detections that are close to each other bsed on timestamp
- However, in this process along with decrease in FP, we have more False Negatives i.e. some anomalies are not detected. 

TODO:
- change deduplication stratergy, if possible